get as many metrics as possible, require neighborhood, # rooms, etc, take random ratings and rating numbers a few times and find a range via the predictions. 

categories are put in alphabetical order, then the first one is dropped. 


In [2]:
import pandas as pd
#import matplotlib.pyplot as plt
import numpy as np
#import seaborn as sns
#import scipy
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
import dill as pickle
#%matplotlib inline

july = pd.read_csv('./data/tomslee_airbnb_new_york_1438_2017-07-12.csv')
july = july.drop(['room_id','survey_id', 'host_id', 'country', 'city', 'bathrooms', 'minstay', 'last_modified', 'location', 'name', 'latitude', 'longitude'], 1)
#remove listings we already know are outliers (price greater than 500 and rooms at 50)
july = july[(july.price < 500) & (july.bedrooms < 50) & (july.overall_satisfaction >= 3)]
july.head()

/Users/claireramming/.virtualenvs/retool-cs/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  if sys.path[0] == '':


,room_type,borough,neighborhood,reviews,overall_satisfaction,accommodates,bedrooms,price,property_type
0,Shared room,Queens,Jackson Heights,6,5.0,4,1.0,85.0,House
1,Shared room,Brooklyn,Cypress Hills,14,4.0,6,1.0,90.0,House
2,Shared room,Brooklyn,Sheepshead Bay,3,4.5,6,1.0,239.0,Apartment
3,Shared room,Manhattan,Hell's Kitchen,35,5.0,2,1.0,165.0,Apartment
7,Shared room,Manhattan,Harlem,8,5.0,2,1.0,110.0,Apartment


In [2]:
july.room_type.unique()

array(['Shared room', 'Entire home/apt', 'Private room'], dtype=object)

In [3]:
july.property_type.value_counts()

Apartment             19814
House                  2372
Loft                    503
Townhouse               410
Condominium             221
Other                   100
Bed & Breakfast          96
Guesthouse               32
Hostel                   15
Villa                    14
Guest suite              13
Dorm                     12
Bungalow                 11
Boutique hotel            8
Timeshare                 6
Boat                      4
Cabin                     3
Vacation home             2
In-law                    2
Serviced apartment        2
Chalet                    2
Camper/RV                 2
Earth House               1
Castle                    1
Casa particular           1
Name: property_type, dtype: int64

In [3]:
prop_dict = {}
for prop_type in july.property_type.unique():
    prop_dict[prop_type] = 'Other'

In [4]:
prop_dict = {}
for prop_type in july.property_type.unique():
    prop_dict[prop_type] = 'Other'

prop_dict['Apartment'] = 'Apartment'
prop_dict['Bed & Breakfast'] = 'Bed & Breakfast'
prop_dict['Serviced Apartment'] = 'Apartment'
prop_dict['Casa particular'] = 'Bed & Breakfast'
prop_dict['Loft'] = 'Loft'
prop_dict['Townhouse'] = 'Townhouse'
prop_dict['Condominium'] = 'Condominium'
prop_dict['House'] = 'House'

july.property_type.replace(prop_dict, inplace=True)

In [6]:
#setting up sorted arrays so position is easy to find
properties = np.sort(july.property_type.unique())
neighbs = np.sort(july.neighborhood.unique())
rooms = np.sort(july.room_type.unique())
bors = np.sort(july.borough.unique())

sorted_arrays = [properties, neighbs, rooms, bors]

#uncomment below to save to pickle
# filename = 'sorted_arrays.pk'
# with open(filename, 'wb') as file:
#    pickle.dump(sorted_arrays, file)

In [7]:
#setting up arrays that will be put together at the end to make X for prediction
property_types = np.zeros((10, len(july.property_type.unique())-1))
neighborhoods = np.zeros((10, len(july.neighborhood.unique())-1))
boroughs = np.zeros((10, len(july.borough.unique())-1))
room_types = np.zeros((10, len(july.room_type.unique())-1))

feature_arrays = [property_types, neighborhoods, boroughs, room_types]

#uncomment below to save to pickle
filename = 'feature_arrays.pk'
with open(filename, 'wb') as file:
    pickle.dump(feature_arrays, file)

In [8]:
#define functions

def set_property(prop_in):
    props = np.searchsorted(properties, prop_in)

    if props != 0:
        for i in range(10):
            property_types[i][props-1] = 1
        
def set_neighborhood(neighb_in):
    neighb = np.searchsorted(neighbs, neighb_in)

    if neighb != 0:
        for i in range(10):
            neighborhoods[i][neighb-1] = 1
        
def set_borough(borough_in):
    bor = np.searchsorted(bors, borough_in)

    if bor != 0:
        for i in range(10):
            boroughs[i][bor-1] = 1

def set_room(room_in):
    room = np.searchsorted(rooms, room_in)

    if room != 0:
        for i in range(10):
            room_types[i][room-1] = 1

def build_numerical(beds_in, accom_in):
    array = np.zeros((10,4))
    for i in range(10):
        array[i][3] = int(accom_in) #accomodates
        array[i][2] = int(beds_in) #num of beds
        array[i][1] = (np.random.rand() * 2) + 3 #rating - generate a number between 3 and 5
        array[i][0] = np.random.randint(1, 454) #num of reviews, generate number between 1 and max # of reviews
    return array

def predict_price(prop, neighb, borough, room, room_num, accom_num):   
    set_property(prop)
    set_neighborhood(neighb)
    set_borough(borough)
    set_room(room)
    numerical = build_numerical(room_num,accom_num)

    features = np.concatenate((numerical, room_types, boroughs, neighborhoods, property_types), 1)
    y_pred = loaded_model.predict(features)
    print(np.mean(y_pred).round(0))

In [9]:
#set up input
set_property("House")
set_neighborhood("Brooklyn")
set_borough("Park Slope")
set_room("Entire home/apt")
numerical = build_numerical(1,4)

features = np.concatenate((numerical, room_types, boroughs, neighborhoods, property_types), 1)

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.pipeline import make_pipeline

july = july[july.overall_satisfaction >= 3]
july_y = july.price
july_dumb = july.drop('price',1)
july_x = pd.get_dummies(july_dumb, drop_first=True)

# Split the data into a training and test set.
Xlr, Xtestlr, ylr, ytestlr = train_test_split(july_x, july_y, test_size=.3, random_state=50)

scaler = preprocessing.StandardScaler()
#X_train = scaler.fit_transform(Xlr)

elnet = ElasticNet()

pipe = make_pipeline(scaler, elnet)

# Fit the model on the trainng data.
pipe.fit(Xlr, ylr)

/Users/claireramming/.virtualenvs/retool-cs/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  import sys


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('elasticnet', ElasticNet())])

In [13]:
def predict_price(prop, neighb, borough, room, room_num, accom_num):   
    set_property(prop)
    set_neighborhood(neighb)
    set_borough(borough)
    set_room(room)
    numerical = build_numerical(room_num,accom_num)

    features = np.concatenate((numerical, room_types, boroughs, neighborhoods, property_types), 1)
    y_pred = loaded_model.predict(features)
    average = np.mean(y_pred).round(0)
    return average

In [22]:
predict_price("Apartment", "Brooklyn", "Park Slope", "Entire home/apt", 1, 4)

/Users/claireramming/.virtualenvs/retool-cs/lib/python3.7/site-packages/sklearn/base.py:442: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


153.0

In [23]:
#html widget, try bokeh, holoviews, plotly, shiny 
#residuals

In [ ]:
tup =[(neighb, neighb) for neighb in july.neighborhood.unique()]

In [ ]:
tup

In [19]:
#save to pickle
filename = './pickled/model_v1.pk'

with open(filename, 'wb') as file:
    pickle.dump(pipe, file)

In [20]:
#open from pickle (moved to 'functions' folder after saving)
filename = 'pickled/model_v1.pk'
with open(filename ,'rb') as f:
    loaded_model = pickle.load(f)

In [6]:
filename = 'functions/sorted_arrays.pk'
with open(filename ,'rb') as f:
    sorted_arrays = pickle.load(f)

In [7]:
filename = 'functions/feature_arrays.pk'
with open(filename ,'rb') as f:
    feature_arrays = pickle.load(f)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import scipy
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
import dill as pickle
%matplotlib inline

In [8]:
#initialize objects from pickled files

#sorted arrays
properties = sorted_arrays[0]
neighbs = sorted_arrays[1]
rooms = sorted_arrays[2]
bors = sorted_arrays[3]

#feature arrays
property_types = feature_arrays[0]
neighborhoods = feature_arrays[1]
boroughs = feature_arrays[2]
room_types = feature_arrays[3]